In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
from abduction_memorability.event import Event, Label
from abduction_memorability.abduction_module import SurpriseAbductionModule
from abduction_memorability.predicate import Predicate
from abduction_memorability.memory import Memory
from abduction_memorability.predicate_filter import OptimizedFilter
from abduction_memorability.helpers import Helpers

from typing import Tuple
import random
import datetime as dt

import plotly.io as pio
import plotly.express as px
import plotly.offline as py

In [3]:
root_label = Label()
day_label = Label(name="day", parent=root_label)

all_events: list[Event] = []
MARIAGE_DAY = 120
NOW = 1000
MARIAGE_DAY_CPLX = 4 # Score computed from a first pass (without the DaysFrom predicate)

In [4]:
all_events.clear()
for day in range(0, NOW + 1):
    event = Event(
        label=day_label,
        timestamp = day,
        duration=1,
        characteristics=dict(
            mariage = day == MARIAGE_DAY
        )
    )
    all_events.append(event)
print(len(all_events))
mem = Memory(all_events)

1001


In [5]:
class Mariage(Predicate):
    def __init__(self, mem, prog):
        super().__init__(mem, prog)
    
    def __call__(self, event: Event):
        if self._prog > 1:
            return None
        return event.get_char("mariage")

    def __str__(self):
        return "Mariage()"

class DayAgo(Predicate):
    def __init__(self, mem, prog):
        super().__init__(mem, prog)

    def __call__(self, event: Event):
        if self._prog > NOW + 1:
            return None
        return event.timestamp == NOW - self._prog

    def __str__(self):
        return f'DaysAgo({self._prog})'

    def program_length(self):
        return Helpers.bit_length(self._prog, use_floor=False)

class DaysFrom(Predicate):
    """Predicate to describe a day wrt another day from the memory.

    The program length of this predicate is a bit tricky to define: to simplify computations in this example, it can only work as a depth 1: it takes the 
    """
    def __init__(self, mem, prog):
        super().__init__(mem, prog)
        self.days = prog - MARIAGE_DAY

    def __call__(self, event):
        if abs(self.days) > NOW:
            return None
        return event.timestamp - MARIAGE_DAY == self.days

    def program_length(self):
        return 1 + Helpers.bit_length(abs(self.days), use_floor=False) + MARIAGE_DAY_CPLX

In [6]:
module = SurpriseAbductionModule(
    mem,
    predicates=[
        DayAgo,
        Mariage,
        DaysFrom
    ],
    max_depth = 2
)

Loaded the memory with 1001 items!
Computing complexities with 2 passes
Starting pass 0 with 1 memories to explore
Finished pass 0 in 0.8659422397613525s.
Improved complexity for 1128 event(s)
Computing memorability scores for all events !


In [7]:
df = module.dataframe_output()
df
df[df["time"] == 120]

,id,time,label,complexity,recipe,memorability,date
120,120,120,event/day,4.584963,[Mariage()],14.411588,1970-01-01 00:02:00


In [8]:
fig = px.scatter(df, x="time", y="memorability", hover_data=['recipe'], height=800)
fig.update_traces(marker={'size':10})
fig.update_layout(
    showlegend=False,
    font=dict(
        size=25
    ),
    xaxis_title="Days"
)
fig.add_shape(
    type="circle",
    xref="x",
    yref="y",
    x0 = MARIAGE_DAY - 6,
    y0 = 15.4,
    x1 = MARIAGE_DAY + 6,
    y1 = 15.9,
)

In [9]:
fig2 = px.scatter(df, x="time", y="complexity", hover_data=['recipe'], height=800)
fig2